In [1]:
import numpy
# scipy.special for the sigmoid function expit()
import scipy.special
# library for plotting arrays
import matplotlib.pyplot
# ensure the plots are inside this notebook, not an external window
%matplotlib inline
import imageio
import glob

In [2]:
# neural network class definition
class neuralNetwork:
    
    
    # initialise the neural network
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # link weight matrices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc 
        self.wih = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))

        # learning rate
        self.lr = learningrate
        
        # activation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass

    
    # train the neural network
    def train(self, inputs_list, targets_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        # output layer error is the (target - actual)
        output_errors = targets - final_outputs
        # hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hidden_errors = numpy.dot(self.who.T, output_errors) 
        
        # update the weights for the links between the hidden and output layers
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        
        # update the weights for the links between the input and hidden layers
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        
        pass

    
    # query the neural network
    def query(self, inputs_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

In [3]:
# number of input, hidden and output nodes# numbe 
input_nodes = 4
hidden_nodes = 50
output_nodes = 3

# learning rate
learning_rate = 0.2

# create instance of neural network
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)

In [4]:
# load the mnist training data CSV file into a list
training_data_file = open("iris_train.csv", 'r')
training_data_list = training_data_file.readlines()[1:]
training_data_file.close()

In [5]:
# train the neural network

# epochs is the number of times the training data set is used for training
epochs = 150

for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = numpy.asfarray(all_values[1:5])
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[5])] = 0.99
        n.train(inputs, targets)
        pass
    pass

In [6]:
# mnist 테스트 데이터인 csv 파일을 리스트로 불러오기
test_data_file = open("iris_test.csv", 'r')
test_data_list = test_data_file.readlines()[1:]
test_data_file.close()
test_data_list

['1,5.1,3.5,1.4,0.2,0\n',
 '2,4.9,3,1.4,0.2,0\n',
 '3,4.7,3.2,1.3,0.2,0\n',
 '51,7,3.2,4.7,1.4,1\n',
 '52,6.4,3.2,4.5,1.5,1\n',
 '53,6.9,3.1,4.9,1.5,1\n',
 '101,6.3,3.3,6,2.5,2\n',
 '102,5.8,2.7,5.1,1.9,2\n',
 '103,7.1,3,5.9,2.1,2\n',
 '104,6.3,2.9,5.6,1.8,2\n',
 '12,4.8,3.4,1.6,0.2,0\n',
 '13,4.8,3,1.4,0.1,0\n',
 '14,4.3,3,1.1,0.1,0\n',
 '15,5.8,4,1.2,0.2,0\n',
 '16,5.7,4.4,1.5,0.4,0\n',
 '66,6.7,3.1,4.4,1.4,1\n',
 '67,5.6,3,4.5,1.5,1\n',
 '68,5.8,2.7,4.1,1,1\n',
 '69,6.2,2.2,4.5,1.5,1\n',
 '70,5.6,2.5,3.9,1.1,1\n',
 '116,6.4,3.2,5.3,2.3,2\n',
 '117,6.5,3,5.5,1.8,2\n',
 '118,7.7,3.8,6.7,2.2,2\n',
 '119,7.7,2.6,6.9,2.3,2\n',
 '120,6,2.2,5,1.5,2\n']

In [7]:
# 신경망 테스트하기

# 신경망의 성능의 지표가되는 성적표를 아무 값도 가지지 않도록 초기화
scorecard = []

# 테스트 데이터 모음 내의 모든 레코드 탐색
for record in test_data_list:
    # 레코드를 쉼표에 의해 분리
    all_values = record.split(',')
    # 정답은 첫 번째 값
    correct_label = int(all_values[5])
    # 입력 값의 범위와 값 조정
    inputs = numpy.asfarray(all_values[1:5]) 
    # 신경망에 질의
    outputs = n.query(inputs)
    # 가장 높은 값의 인덱스는 레이블의 인덱스와 일치
    label = numpy.argmax(outputs)
    # 정답 또는 오답을 리스트에 추가
    if (label == correct_label):
        # 정답인 경우 성적표에 1을 더함
        scorecard.append(1)
        print("label is : ", label)
        print("correct label is : ", correct_label)
    else:
        # 정답이 아닌 경우 성적표에 0을 더함
        scorecard.append(0)
        print("label is : ", label)
        print("correct label is : ", correct_label)
        pass
    pass

label is :  0
correct label is :  0
label is :  0
correct label is :  0
label is :  0
correct label is :  0
label is :  1
correct label is :  1
label is :  1
correct label is :  1
label is :  1
correct label is :  1
label is :  2
correct label is :  2
label is :  2
correct label is :  2
label is :  2
correct label is :  2
label is :  2
correct label is :  2
label is :  0
correct label is :  0
label is :  0
correct label is :  0
label is :  0
correct label is :  0
label is :  0
correct label is :  0
label is :  0
correct label is :  0
label is :  1
correct label is :  1
label is :  2
correct label is :  1
label is :  1
correct label is :  1
label is :  2
correct label is :  1
label is :  1
correct label is :  1
label is :  2
correct label is :  2
label is :  2
correct label is :  2
label is :  2
correct label is :  2
label is :  2
correct label is :  2
label is :  2
correct label is :  2


In [8]:
# 정답의 비율인 성적을 계산해 출력
scorecard_array = numpy.asarray(scorecard)
print ("performance = ", scorecard_array.sum() / scorecard_array.size)

performance =  0.92
